# DAP CLIENT

The `Dap Client` plays a pivotal role in the interaction between the client and the server, acting as a bridge that enables the communication between the user and the `DAP Server`. Its primary responsibilities and functionalities include:

1. **Establishing Connection:**
   The client initiates and maintains a stable connection to the server, facilitating the exchange of information between the user and the server. It uses the `socket` library to form this connection, specifying the hostname and port number of the server.

2. **Sending Requests:**
   The client has the capability to format and send various requests to the `DAP Server`. These requests can range from initializing the debugger with a specific executable path, setting arguments and breakpoints, to starting the debugger and executing other debugger commands.

3. **Receiving Responses:**
   Post the dispatch of requests, the client eagerly awaits the server’s response. It interprets the incoming messages, decoding them from their JSON format and presenting them to the user in a readable and understandable format.

4. **Handling Timeouts:**
   The client is equipped to manage scenarios where the server response is delayed or not received within a specified timeframe, ensuring that the user is informed of any delays or issues in the communication process.

5. **Closing Connection:**
   Upon completion of the debugging session or when the user decides to terminate the session, the client efficiently closes the connection with the server, ensuring that all resources are released appropriately.

6. **Resetting Connection:**
   The client can reset the connection if needed, allowing users to restart their debugging sessions without manual interruptions or reconfigurations.

### Importance:
The manifestation of the debugger client, as illustrated, does not have to be rigid in its form; its essence lies in its protocol adherence rather than its structural embodiment. Any programs capable of sending and dispatching JSON-RPC messages, conforming to the socket-based wire protocol, can essentially assume the role of a debugger client. This inherent flexibility means that a wide spectrum of applications, tools, or interfaces, regardless of their design paradigms or operational contexts, can interact with the server, exchanging information seamlessly. This means tools like IDE's (Visual Studio, IntelliJ, etc) can conform to this by allowing for a method to send and receive the DAP communications.

It’s this adaptability in communication that enables diverse and disparate systems to utilize, send, and receive data, fostering an inclusive ecosystem where varied components can coalesce to leverage the functionalities provided by the DebugServer. The universal language of JSON-RPC over sockets ensures that the dialogue between client and server remains coherent and standardized, granting different implementations the liberty to focus on their unique value propositions while maintaining interoperability.

In [1]:
import json
import socket
import time
import random
from pprint import pprint


class DebuggerConnection:
    def __init__(self, hostname='localhost', port=12345, timeout=5):
        self.msg_id = random.randint(1, 500)  # Initialize with a random value
        self.connection_no = 1
        self.mysocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.mysocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.port = port
        print((port))
        self.hostname = hostname
        self.mysocket.settimeout(timeout)
        self.connect()

    def connect(self):
        self.mysocket.connect((self.hostname, self.port))
        self.connection_no += 1
        time.sleep(1)
        self.recvMessage("connection no {} ".format(self.connection_no))

    def parse_all_messages(self, msg_bytes):
        msg_str = msg_bytes.decode('utf-8')
        split_delimiter = 'Content-Length: '
        msg_list = msg_str.split(split_delimiter)
        if msg_list[0] == '':
            msg_list = msg_list[1:]
        parsed_msg_list = [msg.split('\r\n\r\n') for msg in msg_list]
        for index in range(len(parsed_msg_list)):
            parsed_msg_list[index][0] = split_delimiter + parsed_msg_list[index][0]
        return parsed_msg_list

    def recvMessage(self, func_name=None):
        print("Receiving..")
        try:
            recv_data = self.mysocket.recv(1024 * 1024)
            if recv_data:
                list_of_msgs = self.parse_all_messages(recv_data)
                for msg in list_of_msgs:
                    print("Received data:")
                    print(msg[0])
                    try:
                        pprint(json.loads(msg[1]))
                    except:
                        print("json.loads failed in recvMessage, printing entire message")
                        print(msg)
                    print()
        except socket.timeout:
            msg = "Timeout. Nothing was received."
            if func_name:
                msg += f" at {func_name}"
            print(msg)
            return

    def sendMessage(self, msg):
        s_msg = json.dumps(msg)
        data = 'Content-Length: {0}\r\n\r\n{1}'.format(len(s_msg), s_msg)
        print(f'Sending data: Content-Length: {len(s_msg)}')
        pprint(msg)
        print()
        self.mysocket.send(data.encode())
        time.sleep(1)

    def close_connection(self):
        self.mysocket.close()

    def reset_connection(self):
        self.close_connection()
        self.mysocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.connect()


class Debugger:
    def __init__(self, connection, filename=None):
        self.connection = connection
        self.dbgfile = filename

    def attach_to_server(self):
        self.doRequest( 
            {'command': 'attach',                                                      
             'arguments': {                                                           
               'name': 'client_class',                                                         
             },  
            } 
        )                                                                          
        self.connection.recvMessage("attach")

    def initialize(self, clientID=None):
            if clientID is None:
                clientID = self.__class__.__name__
            self.doRequest( {                                                                    
                'command': 'initialize',                                                  
                'arguments': {                                                            
                    'adapterID': 'TESTCLIENT',
                    'clientID': clientID,
                    'clientName':'GDBClientSide.py',                                                  
                    'linesStartAt1': True,                                                  
                    'columnsStartAt1': True,                                                
                    'pathFormat': 'path',
                    'supportsVariableType': True,
                    'supportsVariablePaging': False,
                    'supportsRunInTerminalRequest': True
                },                                                                        
            } )
            self.connection.recvMessage("initialize")

    def doRequest(self, msg):
        this_id = self.connection.msg_id
        self.connection.msg_id += 1
        msg['seq'] = this_id
        msg['type'] = 'request'
        self.connection.sendMessage(msg)
        return self.connection.recvMessage()
    
    def set_breakpoints(self, line_list):
        bp_list = []
        for line in line_list:
            bp_list.append({'line':line})
        
        self.initialize()
        
        self.doRequest( {                                                                    
              'command': 'setBreakpoints',                                              
              'arguments': {                                                            
                'source':{'path': self.dbgfile}, #could alternately give sourceReference.
                'breakpoints':bp_list,                                                                                                        
              },                                                                        
        } )
        self.connection.recvMessage("setBreakpoints")
        
        self.configurationDone()

    def configurationDone(self):
        self.doRequest( {                                                                    
              'command': 'configurationDone',                                                                                                                         
            } ) 
        self.connection.recvMessage("configurationDone")
        
    def pause(self, threadID=1):
        #The pause request suspends the debuggee.
        #Not sure when to use it
        self.doRequest( {                                                                    
              'command': 'pause',                                                  
              'arguments': {                                                            
                'threadId': threadID,                                                  
              },                                                                        
            } )                                                                         
        self.connection.recvMessage("pause")
    
    def next_step(self, threadID=1):
        #Only makes sense when execution of the debuggee is paused.
        #executes one step and then pauses again
        self.doRequest( {                                                                    
            'command': 'next',
                'arguments': {                                                            
            'threadId': threadID,                                                          
            },                                                                        
        } )
        self.connection.recvMessage("next")
    
    def continue_execution(self, threadID=1):
        #Only makes sense when execution of the debuggee is paused.
        #executes one step and then pauses again
        self.doRequest( {                                                                    
            'command': 'continue',
                'arguments': {                                                            
            'threadId': threadID,                                                          
            },                                                                        
        } )
        self.connection.recvMessage("continue")
    
    def get_threads(self):
        self.DoRequest( {                                                                    
          'command': 'threads',                                                     
        } )
        self.connection.recvMessage("threads")
    
    def get_stack_trace(self, threadID=1):
        self.doRequest( {                                                                    
          'command': 'stackTrace',
                'arguments': {                                                            
            'threadId': threadID,                                                          
          },                                                                        
        } )
        self.connection.recvMessage("stackTrace")
    
    def get_scopes(self, frameID):
        self.doRequest( {                                                                    
          'command': 'scopes',
            'arguments': {                                                            
            'frameId': frameID,                                                          
          },                                                                        
        } )
        self.connection.recvMessage("scopes")
    
    def get_variables(self, variable_ref):
        self.doRequest( {                                                                    
            'command': 'variables',
                'arguments': {                                                            
                    'variablesReference': variable_ref,
          },                                                                        
        } )
        self.connection.recvMessage("variables")  
        
    def step_in(self, threadID=1):
        self.doRequest( {                                                                    
            'command': 'stepIn',
                'arguments': {                                                            
                    'threadId': threadID,
                },                                                                        
        } )
        self.connection.recvMessage("stepIn")
    
    def step_out(self, threadID=1):
        self.doRequest( {                                                                    
            'command': 'stepOut',
                'arguments': {                                                            
                    'threadId': threadID,
                },                                                                        
        } )
        self.connection.recvMessage("stepOut")
    
    def read_server(self, cleanup=False):
        if cleanup:
            self.connection.close_connection()
        else:
            self.connection.recvMessage("End of execution")


{'command': 'initialize', 'path_to_executable': './helloworld', 'seq': 1, 'type': 'request'}
Receiving..
Timeout. Nothing was received.
{'command': 'setArgs', 'args': 'Hello World', 'seq': 2, 'type': 'request'}
Receiving..
Timeout. Nothing was received.
{'command': 'setBreakpointAtLine', 'line': 22, 'seq': 3, 'type': 'request'}
Receiving..
Timeout. Nothing was received.
{'command': 'start', 'seq': 4, 'type': 'request'}
Receiving..
Timeout. Nothing was received.


### Example Usage: DebuggerClient with `helloworld`

This code snippet demonstrates the utilization of `DebuggerClient` to initialize and interact with a simple program, `helloworld`.

#### Initialization:
Here, the client establishes a connection and transmits an `initialize` command, defining the path to the `helloworld` executable.

#### Setting Arguments:
Subsequently, the `setArgs` command is dispatched to supply "Hello World" as an argument to the program.

#### Placing a Breakpoint:
The `setBreakpointAtLine` command is utilized to institute a breakpoint at line `22`.

#### Execution Commencement:
The `start` command is propagated to inaugurate the execution of the program, which will suspend at the appointed breakpoint.

#### Terminating Connection:
Post the completion of the desired interactions, the `close_connection()` method is invoked to terminate the connection.

---
**Note:** The range and scope of interactions aren't limited to the demonstrated commands. Depending on the functionalities implemented on the server side, a variety of additional commands can be incorporated, thus broadening the array of feasible interactions and capabilities provided by the server.


In [ ]:
conn = DebuggerConnection()

# Usage example
debugger = Debugger(conn)
debugger.doRequest({"command": "initialize", "path_to_executable": "./helloworld"})
debugger.doRequest({"command": "setArgs", "args": "Hello World"})
debugger.doRequest({"command": "setBreakpointAtLine", "line": 22})
debugger.doRequest({"command": "start"})
conn.close_connection()

### Example Usage: DebuggerClient with Python program `helloworld.py`

This code snippet demonstrates the utilization of `DebuggerClient` to initialize and interact with a simple python program, `helloworld.py`.

Debugpy relies on a different method of attaching itself onto binaries, so a seperate implementation has been made for simplicity and clarity.

In [ ]:
conn = DebuggerConnection()

# Usage example
debugger = Debugger(conn, 'hellouser.py')
debugger.initialize()
debugger.attach_to_server()
debugger.set_breakpoints([10])
debugger.pause()
debugger.next_step()

conn.close_connection()